In [ ]:
import os
import shutil
from google.colab import drive

# 1. Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# 2. Define source path in MyDrive
# IMPORTANT: Please update this path to where your 'linemod_preprocessed.zip' file is located in your Google Drive.
source_file_path = '/content/drive/MyDrive/Linemod_preprocessed.zip' # Example path, adjust as needed

# 3. Define destination path
destination_dir = '/content/6D_Pose_Estimation_light/dataset/'

# Ensure destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# 4. Unzip the file directly to the destination directory
if os.path.exists(source_file_path):
    print(f"Unzipping '{source_file_path}' directly to '{destination_dir}'...")
    # Use !unzip with -q for quiet output and -d to specify destination directory
    !unzip -q "{source_file_path}" -d "{destination_dir}"
    print("File unzipped successfully.")
else:
    print(f"Error: Source file not found at '{source_file_path}'.")
    print("Please ensure the 'linemod_preprocessed.zip' file exists in your Google Drive at the specified path, or update 'source_file_path'.")

print(f"\nContents of the destination folder in '{destination_dir}':")
!ls -F {destination_dir}

In [ ]:
import yaml

real_classes = {
    0: '01',
    1: '02',
    2: '04',
    3: '05',
    4: '06',
    5: '08',
    6: '09',
    7: '10',
    8: '11',
    9: '12',
    10: '13',
    11: '14',
    12: '15'
}

models_info = "/content/6D_Pose_Estimation_light/dataset/Linemod_preprocessed/models/models_info.yml"

# Lettura del file YAML
with open(models_info, 'r') as f:
    models_data = yaml.safe_load(f)

# Popolazione del dizionario diam_pose
diam_pose = {}
for key, class_id in real_classes.items():
    if int(class_id) in models_data:
        diam_pose[key] = models_data[int(class_id)]['diameter']


In [ ]:
# Notebook cell: load YOLO (finetuned), RotationNet (checkpoint), pinhole and PoseEvaluator for LINEMOD ADD eval
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from ultralytics import YOLO                    # pip install ultralytics
from models.RotationNet import RotationNet           # replace with your RotationNet implementation import
from utils.pinholeFunction import get_3d_coordinates                    # replace with your pinhole function import
from utils.PoseEvaluator import PoseEvaluator            # replace with your PoseEvaluator import
from utils.posenet_dataset_ALL import LineModPoseDataset as LinemodDataset  # replace with your dataset import
from utils.utils_geometric import crop_square_resize, quaternion_to_matrix   # replace with your crop function import
import numpy as np

# --- Paths (replace with actual files) ---
YOLO_WEIGHTS = "checkpoints\\yolo\\best.pt"
ROTNET_CHECKPOINT = "checkpoints\\rotationNet\\best_posenet.pth"
LINEMOD_ROOT = "/path/to/linemod"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# --- Load models ---
yolo = YOLO(YOLO_WEIGHTS).to(DEVICE)

rnet = RotationNet().to(DEVICE)  # adjust constructor args as needed
ckpt = torch.load(ROTNET_CHECKPOINT, map_location="cpu")
state = ckpt.get("state_dict", ckpt)
rnet.load_state_dict(state)
rnet.eval()

# --- Dataset and evaluator ---
dataset = LinemodDataset(LINEMOD_ROOT, split="eval")  # adjust API as needed
evaluator = PoseEvaluator()  # adjust API as needed

val_loader = DataLoader(dataset, batch_size=32, shuffle=False)
# --- Helper: convert RotationNet output to (R,t) pose (implement for your model) ---


# --- Run evaluation loop (example; adapt to your dataset/model APIs) ---
for sample in val_loader:
    img = sample["path"].to(DEVICE)  # input image     
    gt_t = sample.get("t").to(DEVICE)  # ground-truth poses per object
    gt_R = np.array(sample.get("R")).reshape(3, 3).to(DEVICE)
    obj_id = sample.get("obj_id").to(DEVICE)

    # Run YOLO on the image and normalize results into a plain list `detections`
    results = yolo(img).to(DEVICE)            # ultralytics Results
    detections = []
    if len(results) > 0:
        boxes = results[0].boxes
        # boxes.xyxy is (N,4); boxes.conf and boxes.cls provide confidence and class
        xywh = boxes.xywh
        try:
            confs = boxes.conf
        except Exception:
            confs = np.zeros(len(xywh))
        try:
            clss = boxes.cls
        except Exception:
            clss = np.zeros(len(xywh))
        for i in range(len(xywh)):
            x1, y1, w, h = xywh[i]
            conf = float(confs[i]) if len(confs) > i else 0.0
            cls = int(clss[i]) if len(clss) > i else 0
            detections.append(np.array([x1, y1, w, h, conf, cls]))

    preds = []
    for det in detections:
        
        x1, y1, x2, y2, conf, cls = det.tolist()
        if real_classes[int(cls)] == obj_id:
        # convert to x,y,w,h for crop function
            bbox_xywh = [x1, y1, w, h]
            crop = crop_square_resize(img, bbox_xywh)
            # preprocess crop as required by RotationNet

            with torch.no_grad():
                rot_out = rnet(crop)
            R = quaternion_to_matrix(rot_out)
            t = get_3d_coordinates
            preds.append({"obj_id": int(cls), "R": R, "t": t})

    evaluator.add(gt_poses=gt_poses, pred_poses=preds, obj_ids=obj_ids)

print("ADD metric:", evaluator.compute())